# Used Car Price Predictions

Tutorial: [Gabriel Atkin](https://www.youtube.com/watch?v=bFKuw3JlvCI&ab_channel=GabrielAtkin)

Dataset: [Used Cars Dataset](https://www.kaggle.com/austinreese/craigslist-carstrucks-data)

## Importing Modules

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
import pandas as pd 
import numpy as np 

# For kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/craigslist-carstrucks-data/vehicles.csv


In [2]:
df = pd.read_csv('/kaggle/input/craigslist-carstrucks-data/vehicles.csv')

In [3]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [4]:
# To make the columns is printed all 
# pd.set_option('max_columns', None) # write this in console

In [5]:
df

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

## Preprocessing

In [7]:
df.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [8]:
df.isna().mean()

id              0.000000
url             0.000000
region          0.000000
region_url      0.000000
price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
condition       0.407852
cylinders       0.416225
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
VIN             0.377254
drive           0.305863
size            0.717675
type            0.217527
paint_color     0.305011
image_url       0.000159
description     0.000164
county          1.000000
state           0.000000
lat             0.015342
long            0.015342
posting_date    0.000159
dtype: float64

In [9]:
df.isna().mean() > 0.25

id              False
url             False
region          False
region_url      False
price           False
year            False
manufacturer    False
model           False
condition        True
cylinders        True
fuel            False
odometer        False
title_status    False
transmission    False
VIN              True
drive            True
size             True
type            False
paint_color      True
image_url       False
description     False
county           True
state           False
lat             False
long            False
posting_date    False
dtype: bool

In [10]:
# Drop all null columns
null_columns = df.columns[df.isna().mean() > 0.25]
df1 = df.drop(null_columns, axis=1)

In [11]:
df1.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer', 'title_status', 'transmission', 'type',
       'image_url', 'description', 'state', 'lat', 'long', 'posting_date'],
      dtype='object')

In [12]:
# Drop all uneeded columns
uneeded_columns = ['id', 'url', 'region_url', 'image_url', 'description']
df2 = df1.drop(uneeded_columns, axis=1)

In [13]:
df2.columns

Index(['region', 'price', 'year', 'manufacturer', 'model', 'fuel', 'odometer',
       'title_status', 'transmission', 'type', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [14]:
df2

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,state,lat,long,posting_date
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,maxima s sedan 4d,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,wyoming,28990,2018.0,lexus,es 350 sedan 4d,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [20]:
# Cleaning the data so all null vlaues will be gone 
df3 = df2.dropna(how='any', axis=0) # axis=0 is to remove by rows that has NaN

In [21]:
df3

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,state,lat,long,posting_date
27,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,gas,57923.0,clean,other,pickup,al,32.590000,-85.480000,2021-05-04T12:31:18-0500
28,auburn,22590,2010.0,chevrolet,silverado 1500,gas,71229.0,clean,other,pickup,al,32.590000,-85.480000,2021-05-04T12:31:08-0500
29,auburn,39590,2020.0,chevrolet,silverado 1500 crew,gas,19160.0,clean,other,pickup,al,32.590000,-85.480000,2021-05-04T12:31:25-0500
30,auburn,30990,2017.0,toyota,tundra double cab sr,gas,41124.0,clean,other,pickup,al,32.590000,-85.480000,2021-05-04T10:41:31-0500
31,auburn,15000,2013.0,ford,f-150 xlt,gas,128000.0,clean,automatic,truck,al,32.592000,-85.518900,2021-05-03T14:02:03-0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,maxima s sedan 4d,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,wyoming,28990,2018.0,lexus,es 350 sedan 4d,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [22]:
# Well, there is more columns that need to be cleaned
uneeded_columns_2 = ['manufacturer', 'odometer', 'type', 'posting_date']
df4 = df3.drop(uneeded_columns_2, axis=1)

In [23]:
df4

,region,price,year,model,fuel,title_status,transmission,state,lat,long
27,auburn,33590,2014.0,sierra 1500 crew cab slt,gas,clean,other,al,32.590000,-85.480000
28,auburn,22590,2010.0,silverado 1500,gas,clean,other,al,32.590000,-85.480000
29,auburn,39590,2020.0,silverado 1500 crew,gas,clean,other,al,32.590000,-85.480000
30,auburn,30990,2017.0,tundra double cab sr,gas,clean,other,al,32.590000,-85.480000
31,auburn,15000,2013.0,f-150 xlt,gas,clean,automatic,al,32.592000,-85.518900
...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,maxima s sedan 4d,gas,clean,other,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,s60 t5 momentum sedan 4d,gas,clean,other,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,xt4 sport suv 4d,diesel,clean,other,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,es 350 sedan 4d,gas,clean,other,wy,33.786500,-84.445400


In [25]:
{column: len(df4[column].unique()) for column in df4.columns if df4.dtypes[column] == 'object'}

{'region': 404,
 'model': 18830,
 'fuel': 5,
 'title_status': 6,
 'transmission': 3,
 'state': 51}

In [26]:
# model is too many for categorizing, so it need to be removed to save time
df5 = df4.drop('model', axis=1)

In [27]:
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [28]:
data = onehot_encode(
    df5,
    ['region', 'fuel', 'title_status', 'transmission', 'state'],
    ['reg', 'fuel', 'title', 'trans', 'state']
)

In [29]:
data

,price,year,lat,long,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,reg_altoona-johnstown,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
27,33590,2014.0,32.590000,-85.480000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,22590,2010.0,32.590000,-85.480000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,39590,2020.0,32.590000,-85.480000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,30990,2017.0,32.590000,-85.480000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,15000,2013.0,32.592000,-85.518900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,2019.0,33.786500,-84.445400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
426876,30590,2020.0,33.786500,-84.445400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
426877,34990,2020.0,33.779214,-84.411811,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
426878,28990,2018.0,33.786500,-84.445400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [30]:
for column in data.columns:
    data[column] = data[column].fillna(data[column].mean())

In [32]:
data.isna().sum().sum() # summing all null data if exist by row and columns

0

## Splitting and Scaling

In [33]:
X = data.drop('price', axis=1)
y = data.loc[:, 'price']

In [34]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=34)

## Training

In [36]:
linreg_model = LinearRegression()
linreg_model.fit(X_train, y_train)
linreg_y_preds = linreg_model.predict(X_test)

In [38]:
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    n_estimators=100,
    reg_lambda=1.0
)

lgb_model.fit(X_train, y_train)

lgb_y_preds = lgb_model.predict(X_test)

In [40]:
linreg_loss = np.sqrt(mean_squared_error(y_test, linreg_y_preds))
lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_preds))

In [41]:
print("Linear Regression RMSE:", linreg_loss)
print("Gradient Boosted RMSE:", lgb_loss)

Linear Regression RMSE: 3.4300532437127127e+18
Gradient Boosted RMSE: 4937403.279387117


In [42]:
print("Linear Regression R^2 Score:", linreg_model.score(X_test, y_test))
print("Gradient Boosted R^2 Score:", lgb_model.score(X_test, y_test))

Linear Regression R^2 Score: -8.381341864695994e+23
Gradient Boosted R^2 Score: -0.7366369399209505
